In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import pickle
import pandas as pd
from datasets import Value
from torch.utils.data import DataLoader
import argparse
import os
from pymongo import MongoClient
import IPython
from tqdm import tqdm

In [30]:
client = MongoClient()
db_metoo_tweets = client["jason_twitter"]
metoo_tweets = db_metoo_tweets.tweets

In [31]:
tweet_cursor = metoo_tweets.find({"condemnation_prediction":1})

In [32]:
user_ids = set()

In [33]:
for tweet in tqdm(tweet_cursor):
    user_ids.add(tweet["user"]["id"])


8257732it [23:54, 5757.79it/s]


In [34]:
len(user_ids)

1405212

In [35]:
pf_df = pd.read_csv("../../data/ideology-data/followers_list.csv")

In [36]:
pf_df = pf_df.rename(columns={"Unnamed: 0": "account"})

In [37]:
pf_df

,account,followers_list
0,10168082,"['1648278229558636545', '37696245', '123178873..."
1,104237736,"['1259652728', '1648890787869130753', '1502603..."
2,10615232,"['84143903', '1334695790388932609', '992262732..."
3,1082334352711790593,"['1646356229340446720', '24060272', '164004309..."
4,1099199839,"['1047854881640464385', '1452724337685917696',..."
...,...,...
132,92186819,"['1647146994761101312', '3920764053', '1647747..."
133,926446070812602371,"['1635498467307974657', '924283349727961091', ..."
134,9300262,"['528589465', '1648932804401692673', '83726367..."
135,935368364,"['1479847684219490304', '1511704596800016387',..."


In [38]:
new_rows = []
for idx, row in tqdm(pf_df.iterrows()):
    new_rows.extend({"account" : row["account"]  , "id_str": u} for u in ast.literal_eval(row["followers_list"]))
    
pf_df = pd.DataFrame(new_rows)
pf_df

137it [00:03, 35.42it/s]


,account,id_str
0,10168082,1648278229558636545
1,10168082,37696245
2,10168082,1231788738885361665
3,10168082,1624474085425295360
4,10168082,1331736257530122241
...,...,...
677306,939091,1648726291619749888
677307,939091,1048120681936826369
677308,939091,1648732272730972160
677309,939091,1648731259026145292


In [39]:
# followers_df = pd.read_csv("./460_account_followers.csv")
followers_df = pf_df

In [40]:
followers_df

,account,id_str
0,10168082,1648278229558636545
1,10168082,37696245
2,10168082,1231788738885361665
3,10168082,1624474085425295360
4,10168082,1331736257530122241
...,...,...
677306,939091,1648726291619749888
677307,939091,1048120681936826369
677308,939091,1648732272730972160
677309,939091,1648731259026145292


In [41]:
accounts_who_follow_poli = set(followers_df.id_str)

In [42]:
len(accounts_who_follow_poli)

484775

In [43]:
non_poli_users = []
poli_users = []

for user_id in tqdm(user_ids):
    if user_id in accounts_who_follow_poli:
        poli_users.append(user_id)
    else:
        non_poli_users.append(user_id)
len(non_poli_users)

100%|██████████| 1405212/1405212 [00:00<00:00, 2477940.82it/s]


1405212

In [44]:
users_with_estimate = user_ids.intersection(accounts_who_follow_poli)

In [45]:
len(users_with_estimate)

0

In [46]:
filtered_followers_df = followers_df[followers_df.id_str.isin(user_ids)]

In [47]:
len(set(filtered_followers_df.id_str))

0

In [48]:
unique_user_id_df = pd.DataFrame({"id_str":list(user_ids)})

In [49]:
# unique_user_id_df = unique_user_id_df.rename(columns={"user.id_str":"id_str"})
join_df = pd.merge(followers_df, unique_user_id_df, how="inner", on="id_str")

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
join_df

In [ ]:
adj_matrix = pd.DataFrame(0, index=join_df["id_str"].unique(), columns=join_df["account"].unique())

In [ ]:
adj_matrix